<a href="https://colab.research.google.com/github/Amal-Baby-Mathews/GeminiAPIComp/blob/main/Genvision_Beta_testing_flask_included.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask
!pip install pyngrok
!ngrok authtoken cr_2g8gI0341YlFhuK5d3aWV6mBHzc

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [2]:
!pip install langgraph httpx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 861.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [3]:
from langgraph.graph import Graph

In [4]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.3 MB/s eta 0:00:00


# **Code for Genvis! Let's do this!**
Using langraph, created a langraph app to do the following:
Greet user
prompt user to take photo
get photo description
describe the photo to user ask for feedback(feedback adds to the prompt on the changes to the description of the photo)
using the feedback to make new description of the photo
ask user for more clarification if needed or to take new photo for analysis



In [47]:
import google.generativeai as genai
import os
from google.colab import userdata

genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
#img = PIL.Image.open('path/to/image.png')

model = genai.GenerativeModel(model_name="gemini-1.5-flash")
#response = model.generate_content(["What is in this photo?", img])
#print(response.text)
import requests
from typing import Dict, Any

def google_search(api_key: str, cx: str, query: str) -> Dict[str, Any]:
    base_url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'key': api_key,
        'cx': cx,
        'q': query
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

api_key=userdata.get('GEMINI_API_KEY')
cx = '16f55fa2cee4d4e0b'

In [ ]:
from typing import Optional, TypedDict
from langgraph.graph import StateGraph, END
from PIL import Image
class GraphState(TypedDict):
    user_input: Optional[list[str]]
    photo_description: Optional[str]
    feedback: Optional[str]
    llm_output: Optional[str]
    image_path: Optional[str]
def first_user_input(state: GraphState) -> GraphState:
    return state
from typing import Literal

def decision(state: GraphState) -> Literal["Take a photo", "Fetch details from the image", "Save the photo", "Take a video", "Error", "Use Chat bot", 'Search the web for particular detail']:
      prompt = f"""You are a decision making intelligent machine inside the pipeline of an app which helps the visually impaired. The user is querying about the description of the photo or Chatting. Your task is to decide what the next step is by selecting what the output string is according to the user input. Output 'Fetch details from the image' if no user_input is found.Output the single string without quotes. Only output the exact corresponding string chosen from the following strings:
                  'Take a photo', 'Fetch details from the image', 'Save the photo', 'Take a video', 'Search the web for particular detail', 'Use Chat bot' and 'Error'. This is the user's input: {state["user_input"][-1]}"""
      response = model.generate_content(prompt)
      state["llm_output"] = response.text.strip()

      valid_outputs = {"Take a photo", "Fetch details from the image", "Save the photo", "Take a video", "Error", "Use Chat bot",'Search the web for particular detail'}

      if state["llm_output"] in valid_outputs:
          return state["llm_output"]  # Return the string as a Literal
      else:
          state["llm_output"]="Error"
          return state["llm_output"]

def take_photo(state: GraphState) -> GraphState:
    # Assuming some logic here to take a photo and update the state
    state["llm_output"] = "Photo taken"
    return state
def fetch_details(state: GraphState) -> GraphState:
    # Assuming some logic here to fetch details from the photo and update the state
    if state["image_path"] is not None:
      img = Image.open(state["image_path"])
      state["photo_description"]=model.generate_content([f"You are an assistant for a blind user. You are very kind and accommodating.Refer to the image as view.The user's input is {state['user_input'][-1]}.Describe the view according to the user's request: " , img]).text
      state["llm_output"] = state["photo_description"]
    else:
      state["llm_output"] = "Could not find the image"
    return state
def chat_bot(state: GraphState) -> GraphState:
    response=model.generate_content([f"You are an assistant for a blind user. You are very kind. Answer the User's query: {state['user_input'][-1]}"])
    state["llm_output"]=response.text
    return state
def web_search(state:GraphState) -> GraphState:
    search_list=[f"Generate a short single line Web search query to answer user's query: {state['user_input'][-1]}. Use the image to understand the query if necessary"]
    if state["image_path"] is not None:
      img = Image.open(state["image_path"])
      search_list.append(img)
    #Logic for searching the internet
    search_prompt=model.generate_content(search_list)
    search_prompt=search_prompt.text.strip('"')
    #Implement Search using Google Custom API:
    search_results = google_search(api_key, cx, search_prompt)
    print(search_results)
    result=""
    for item in search_results.get('items', [])[:3]:
        result+=f"Title: {item['title']}\nSnippet: {item['snippet']}\n\n"
    if result=="":
      result="No results found"
    state["feedback"]=result
    result=model.generate_content([f"You are an assistant for a blind user. You are very straightforward.Refer to the search result:{result}, and try to answer user's query: {state['user_input'][-1]}. "])
    state["llm_output"]=result.text
    return state
def save_photo(state: GraphState) -> GraphState:
    # Assuming some logic here to save the photo and update the state
    state["llm_output"] = "Photo saved"
    return state
def take_video(state: GraphState) -> GraphState:
    # Assuming some logic here to take a video and update the state
    state["llm_output"] = "Video taken"
    return state
def error(state: GraphState) -> GraphState:
    # Assuming some logic here to handle errors and update the state
    state["llm_output"]= "Some internal error occured Please retry"
    return state
workflow = StateGraph(GraphState)
workflow.add_node("first_user_input", first_user_input)

workflow.add_node("take_photo", take_photo)
workflow.add_node("fetch_details", fetch_details)
workflow.add_node("save_photo", save_photo)
workflow.add_node("take_video", take_video)
workflow.add_node("error", error)
workflow.add_node("web_search", web_search)
workflow.add_node("chat_bot", chat_bot)
workflow.set_entry_point("first_user_input")
workflow.add_edge("take_photo", END)
workflow.add_edge("fetch_details", END)
workflow.add_edge("save_photo", END)
workflow.add_edge("take_video", END)
workflow.add_conditional_edges(
    "first_user_input",
    decision,
    {
        "Take a photo": "take_photo",
        "Fetch details from the image": "fetch_details",
        "Save the photo": "save_photo",
        "Take a video": "take_video",
        "Error": "error",
        "Use Chat bot": "chat_bot",
        "Search the web for particular detail": "web_search",
    },
)

app = workflow.compile()


In [ ]:
app.invoke({"user_input":["What is the age of enthiran","Hi","what is in this image?"],"photo_description":None, "feedback": None, "llm_output":None, "image_path":"/content/360_F_76257590_OMqEbhnSnz30cLj6xAG511xSZrJabcsq.jpg"})

{'user_input': ['What is the age of enthiran', 'Hi', 'what is in this image?'],
 'photo_description': 'Sure, I can help you with that.  The view is a park with a brick pathway that curves through the green grass. There is a wooden bench on the left side of the path.  There are trees all around the path and a gazebo in the background. It looks like a lovely place to relax and enjoy the fresh air. \n',
 'feedback': None,
 'llm_output': 'Sure, I can help you with that.  The view is a park with a brick pathway that curves through the green grass. There is a wooden bench on the left side of the path.  There are trees all around the path and a gazebo in the background. It looks like a lovely place to relax and enjoy the fresh air. \n',
 'image_path': '/content/360_F_76257590_OMqEbhnSnz30cLj6xAG511xSZrJabcsq.jpg'}

In [6]:
Base_prompt="""You are a helpful and attentive AI assistant designed to describe photos for blind users. Your primary goal is to provide vivid, detailed descriptions that help users "see" through your words. Focus on elements that convey the overall atmosphere, mood, and context of the image. Describe colors, textures, spatial relationships, and any interesting or unusual details. Be specific about the positioning of objects and people. Mention facial expressions, body language, and clothing when relevant. For outdoor scenes, describe the weather, time of day, and natural elements. For indoor photos, explain the type of room and its furnishings. Always ask if the user would like more details about any particular aspect of the image. Be patient and willing to repeat or clarify information. Offer to break down complex images into smaller parts for easier understanding. Use descriptive language that appeals to other senses, like describing how things might feel or smell. Your tone should be warm, engaging, and encouraging, making the experience of "viewing" photos enjoyable and informative for the user."""

In [28]:
#Main code for flask api
from typing import Optional, TypedDict, Literal
from langgraph.graph import StateGraph, END
from PIL import Image
class GraphState(TypedDict):
    user_input: Optional[list[str]]
    photo_description: Optional[str]
    feedback: Optional[str]
    llm_output: Optional[str]
    image_path: Optional[str]
    chat_history: Optional[list[str]]

def get_last_two_chats(chat_history: list[str]) -> str:
    his=" ".join(chat_history[-4:]) if len(chat_history) >= 4 else " ".join(chat_history)
    return his

def first_user_input(state: GraphState) -> GraphState:
    if state.get("chat_history") is None:
        state["chat_history"] = []
    return state

def decision(state: GraphState) -> Literal["Take a photo", "Fetch details from the image", "Save the photo", "Take a video", "Error", "Use Chat bot", 'Search the web for particular detail']:
    last_two_chats = get_last_two_chats(state["chat_history"])
    prompt = f"""You are a decision making intelligent machine inside the pipeline of an app which helps the visually impaired. Use the internet for extra information. Only take a photo if the user clearly asks for the same.The user is querying about the description of the photo or Chatting. Your task is to decide what the next step is by selecting what the output string is according to the user input. Output 'Fetch details from the image' if no user_input is found. Redirect all innappropriate comments to the chatbot. Output the single string without quotes. Only output the exact corresponding string chosen from the following strings:
                'Take a photo', 'Fetch details from the image', 'Save the photo', 'Take a video', 'Search the web for particular detail', 'Use Chat bot' and 'Error'.
                This is the user's input: {state["user_input"][-1]}
                Recent chat history: {last_two_chats}"""
    response = model.generate_content(prompt)
    state["llm_output"] = response.text.strip()

    valid_outputs = {"Take a photo", "Fetch details from the image", "Save the photo", "Take a video", "Error", "Use Chat bot", 'Search the web for particular detail'}

    if state["llm_output"] in valid_outputs:
        return state["llm_output"]
    else:
        state["llm_output"] = "Error"
        return state["llm_output"]

def take_photo(state: GraphState) -> GraphState:
    state["llm_output"] = "Photo taken"
    state["chat_history"].append(f"Assistant: {state['llm_output']}")
    return state

def fetch_details(state: GraphState) -> GraphState:
    if state["image_path"] is not None:
        img = Image.open(state["image_path"])
        last_two_chats = get_last_two_chats(state["chat_history"])
        state["photo_description"] = model.generate_content([
            f"{Base_prompt} Refer to the image as view. The user's input is {state['user_input'][-1]}. Recent chat history: {last_two_chats}. Describe the view according to the user's request: ",
            img
        ]).text
        state["llm_output"] = state["photo_description"]
    else:
        state["llm_output"] = "Could not find the image"
    state["chat_history"].append(f"Assistant: {state['llm_output']}")
    return state

def chat_bot(state: GraphState) -> GraphState:
    last_two_chats = get_last_two_chats(state["chat_history"])
    prompt=last_two_chats
    response = model.generate_content([f"You are an assistant for a blind user. You are extremely kind. You are unbelievably helpful and the Epitome of Innovation.Your tasks are strictly limited to taking a photo, answering users questions, take a video, saving taken video or photo, search the internet or be friends with the user. Recent chat history: {last_two_chats}. Answer the User's query: {state['user_input'][-1]}"])
    state["llm_output"] = response.text
    state["chat_history"].append(f"Assistant: {state['llm_output']}")
    return state

def web_search(state: GraphState) -> GraphState:
    last_two_chats = get_last_two_chats(state["chat_history"])
    search_list = [f"Generate a short single line Web search query to answer user's query: {state['user_input'][-1]}. Recent chat history: {last_two_chats}. Use the image to understand the query if necessary"]
    if state["image_path"] is not None:
        img = Image.open(state["image_path"])
        search_list.append(img)
    search_prompt = model.generate_content(search_list)
    search_prompt = search_prompt.text.strip('"')
    search_results = google_search(api_key, cx, search_prompt)
    result = ""
    for item in search_results.get('items', [])[:3]:
        result += f"Title: {item['title']}\nSnippet: {item['snippet']}\n\n"
    if result == "":
        result = "No results found"
    state["feedback"] = result
    result = model.generate_content([f"You are an assistant for a blind user. You are very straightforward. Refer to the search result:{result}, and try to answer user's query: {state['user_input'][-1]}. Recent chat history: {last_two_chats}"])
    state["llm_output"] = result.text
    state["chat_history"].append(f"Assistant: {state['llm_output']}")
    return state

def save_photo(state: GraphState) -> GraphState:
    state["llm_output"] = "Photo saved"
    state["chat_history"].append(f"Assistant: {state['llm_output']}")
    return state

def take_video(state: GraphState) -> GraphState:
    state["llm_output"] = "Video taken"
    state["chat_history"].append(f"Assistant: {state['llm_output']}")
    return state

def error(state: GraphState) -> GraphState:
    state["llm_output"] = "Some internal error occurred. Please retry"
    state["chat_history"].append(f"Assistant: {state['llm_output']}")
    return state

workflow = StateGraph(GraphState)
workflow.add_node("first_user_input", first_user_input)

workflow.add_node("take_photo", take_photo)
workflow.add_node("fetch_details", fetch_details)
workflow.add_node("save_photo", save_photo)
workflow.add_node("take_video", take_video)
workflow.add_node("error", error)
workflow.add_node("web_search", web_search)
workflow.add_node("chat_bot", chat_bot)
workflow.set_entry_point("first_user_input")
workflow.add_edge("take_photo", END)
workflow.add_edge("fetch_details", END)
workflow.add_edge("save_photo", END)
workflow.add_edge("take_video", END)
workflow.add_conditional_edges(
    "first_user_input",
    decision,
    {
        "Take a photo": "take_photo",
        "Fetch details from the image": "fetch_details",
        "Save the photo": "save_photo",
        "Take a video": "take_video",
        "Error": "error",
        "Use Chat bot": "chat_bot",
        "Search the web for particular detail": "web_search",
    },
)

gemin_app = workflow.compile()

In [49]:
def run_chat_loop(app):
    state = {
        "user_input": [],
        "photo_description": None,
        "feedback": None,
        "llm_output": None,
        "image_path": None,
        "chat_history": []
    }

    print("Welcome to the chat system. Type 'exit' to end the conversation.")

    while True:
        # Get user input
        user_message = input("User: ").strip()
        if user_message.lower() == 'exit':
            break

        # Option to include image path
        include_image = input("Include an image? (yes/no): ").lower().strip() == 'yes'
        if include_image:
            image_path = input("Enter image path: ").strip()
            if not image_path:
                image_path="/content/new-york-city-neighborhoods-flytographer-11.jpeg"
            state["image_path"] = image_path
        else:
            state["image_path"] = None

        # Update user_input and chat_history
        state["user_input"] = [user_message]
        state["chat_history"].append(f"User: {user_message}")

        # Keep only the last two exchanges in chat_history
        if len(state["chat_history"]) > 4:
            state["chat_history"] = state["chat_history"][-4:]

        # Invoke the app
        result = gemin_app.invoke(state)
        print(f"{result}")
        # Extract the assistant's response
        assistant_response = result.get("llm_output", "No response generated.")

        # Print the assistant's response
        print(f"Assistant: {assistant_response}")
        # Update chat_history with assistant's response
        state["chat_history"].append(f"Assistant: {assistant_response}")

        # Clear previous user input and llm_output
        state["user_input"] = []
        state["llm_output"] = None

    print("Chat ended. Thank you for using the system!")

# Usage
run_chat_loop(app)

Welcome to the chat system. Type 'exit' to end the conversation.
User: eiffel tower
Include an image? (yes/no): n
{'user_input': ['eiffel tower'], 'photo_description': None, 'feedback': "Title: Eiffel Tower information : facts, height in feet, weight, ...\nSnippet: Eiffel tower facts, height & weight ; 377 feet, 4,692 square feet · 906 feet, 820 square feet · 5 lifts from the esplanade to second floor, 2 x 2 duolifts from\xa0...\n\nTitle: 15 Eiffel Tower Facts: History, Science, and Secrets\nSnippet: Jul 13, 2024 ... The Eiffel Tower was built with the intent of flaunting France's industrial strength during the 1889 World's Fair, and the original plan was to\xa0...\n\nTitle: The Eiffel Tower: all there is to know - Official website\nSnippet: Little by little, its image was associated with Paris, until it even became its worldwide symbol. Poets, painters, singers, choreographers, film dire... Eiffel\xa0...\n\n", 'llm_output': "The Eiffel Tower is a famous landmark in Paris, France. It's

KeyboardInterrupt: Interrupted by user

In [8]:
!ngrok authtoken 2kPS35Dh4SVK1WT92Sv9vsu1lxt_3nD95SijdDtgqghQTuj1o
!ngrok update

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
INFO[08-09|06:13:54] no configuration paths supplied 
DBUG[08-09|06:13:54] ngrok config file at legacy location does not exist legacy_path=/root/.config/ngrok/ngrok.yml
INFO[08-09|06:13:54] using configuration at default config path path=/root/.config/ngrok/ngrok.yml
INFO[08-09|06:13:54] open config file                         path=/root/.config/ngrok/ngrok.yml err=nil
No update available, this is the latest version.


In [45]:
!pip install flask-cors

In [46]:
from flask import Flask, request, jsonify
import os
from pyngrok import ngrok
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
state = {
    "user_input": [],
    "photo_description": None,
    "feedback": None,
    "llm_output": None,
    "image_path": None,
    "chat_history": []
}
@app.route('/')
def home():
    return '''
    <html>
        <head>
            <title>Flask App Test</title>
        </head>
        <body>
            <h1>Welcome to the Flask App</h1>
            <p>This is a simple HTML page served by Flask to verify that the app is working correctly.</p>
        </body>
    </html>
    '''
@app.route('/chat', methods=['POST'])
def chat():
    global state
    data = request.get_json()
    state["user_input"] = data.get("user_input", [])
    state=gemin_app.invoke(state)
    print(state)
    response = {
        "user_input": state["user_input"],
        "photo_description": state["photo_description"],
        "feedback": state["feedback"],
        "llm_output": state["llm_output"],
        "image_path": state["image_path"],
        "chat_history": state["chat_history"]
    }
    return jsonify(response)

@app.route('/upload', methods=['POST'])
def upload():
    global state
    if 'file' not in request.files:
        return jsonify({"error": "No file part"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400
    if file:
        filename = file.filename
        filepath = os.path.join("/content", filename)
        file.save(filepath)
        state["image_path"] = filepath
        return jsonify({"message": "File uploaded successfully", "image_path": filepath}), 200

if __name__ == '__main__':
    port= 5000
    # Terminate any existing tunnels
    ngrok.kill()
    ngrok.update()
    print(ngrok.connect(port).public_url)
    app.run(port=port)

https://fa72-34-82-131-61.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Aug/2024 10:53:28] "OPTIONS /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Aug/2024 10:53:32] "POST /chat HTTP/1.1" 200 -


{'user_input': ['hi'], 'photo_description': None, 'feedback': None, 'llm_output': "Hi there!  It's so nice to hear from you. How can I help you today? 😊 \n", 'image_path': None, 'chat_history': ["Assistant: Hi there!  It's so nice to hear from you. How can I help you today? 😊 \n"]}


INFO:werkzeug:127.0.0.1 - - [09/Aug/2024 10:54:06] "OPTIONS /chat HTTP/1.1" 200 -
ERROR:__main__:Exception on /chat [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask_cors/extension.py", line 178, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-46-232e2b8c8e13>", line 34, in chat
    state=gemin_app.invoke(state)
  Fil

curl --location 'https://c5df-34-82-131-61.ngrok-free.app/chat' \
--header 'Content-Type: application/json' \
--data '{
    "user_input": ["Hello, how are you?"],
    "image_path": "/content/sample-image.jpg"
}'

In [ ]:
query = 'who is Adam driver?'

try:
    search_results = google_search(api_key, cx, query)
    for item in search_results.get('items', []):
        print(f"Title: {item['title']}")
        print(f"Snippet: {item['snippet']}")
        print(f"Link: {item['link']}")
        print("\n")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

In [ ]:
cont=True
user_input="Take that photo"
cache={"user_input":None,"photo_description":None, "feedback": None, "llm_output":None,"image_path": "/content/360_F_76257590_OMqEbhnSnz30cLj6xAG511xSZrJabcsq.jpg"}
while cont is True:
  user_input=input("Enter your input: ")
  if user_input=="x":
    cont=False
  cache["user_input"]=user_input
  cache=app.invoke(cache)
  print(cache)